In [1]:
import subprocess
import sys



In [2]:
# def install(package):
#     subprocess.check_call([sys.executable, "-q", "-m", "pip", "install", package])

# install('awswrangler')
# install('sagemaker')
# install('s3fs')
# install('fsspec')
# install('botocore==1.29.102')


!pip install s3fs
!pip install fsspec
!pip install awswrangler
!pip install sagemaker


  Obtaining dependency information for botocore<2.0.0,>=1.23.32 from https://files.pythonhosted.org/packages/cc/5b/7958e4106d76ef5f27c556c36995437cad0bdec98742bd848a01dc5e16d0/botocore-1.31.76-py3-none-any.whl.metadata
  Using cached botocore-1.31.76-py3-none-any.whl.metadata (6.1 kB)
Using cached botocore-1.31.76-py3-none-any.whl (11.3 MB)
  Attempting uninstall: botocore
    Found existing installation: botocore 1.26.10
    Uninstalling botocore-1.26.10:
      Successfully uninstalled botocore-1.26.10
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.29.41 requires botocore==1.31.41, but you have botocore 1.31.76 which is incompatible.


In [3]:
!pip install botocore==1.26.10


  Using cached botocore-1.26.10-py3-none-any.whl (8.8 MB)
  Attempting uninstall: botocore
    Found existing installation: botocore 1.31.76
    Uninstalling botocore-1.31.76:
      Successfully uninstalled botocore-1.31.76
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.29.41 requires botocore==1.31.41, but you have botocore 1.26.10 which is incompatible.
boto3 1.28.41 requires botocore<1.32.0,>=1.31.41, but you have botocore 1.26.10 which is incompatible.


In [4]:
import pandas as pd
import numpy as np
import datetime as dt
import os              
#(interacting with several os system)

import gc           
#(garbage collector-to free up unused space)

import boto3
import awswrangler as wr
import warnings
import sagemaker
import argparse             
#(to add custom arguments eg print("Hello" + unser input arg (args.name))

import fsspec               
#(provide a unified pythonic interface to local, remote and embedded file systems)

import s3fs                 
#( Pythonic file interface to S3)

from string import Template 
#(template usuing $ based substitution)

#import pandasql


ImportError: cannot import name 'DEPRECATED_SERVICE_NAMES' from 'botocore.docs' (/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/docs/__init__.py)

In [5]:
#import pandasql

# pd.options.display.max_columns=1000
pd.set_option('display.max_rows',1000)
pd.set_option('display.max_columns',100)
warnings.filterwarnings('ignore')

NameError: name 'warnings' is not defined

In [6]:
client = boto3.client('s3')
#path = "s3://intl-euro-uk-datascientist-prod/EU_GHB/health_state_flag/"
#bucket = 'intl-euro-uk-datascientist-prod/EU_GHB/health_state_flag'
prefix = "source"
import boto3
session = boto3.Session(region_name="eu-west-2")

NameError: name 'boto3' is not defined

# Old_Mapping_Table


In [9]:
# def transform_df(claims_raw,elig_df):
   
    
    # import CCS reference table
ccs_icd10_map = wr.s3.read_csv(r's3://intl-euro-uk-datascientist-prod/GHB-reference-table/ccs_dx_icd10cm_2019_1_v2.csv')
ccs_icd10_map = ccs_icd10_map[["'ICD-10-CM CODE'","icd_code","'MULTI CCS LVL 1'","'MULTI CCS LVL 1 LABEL'",\
                                   "'MULTI CCS LVL 2'","adjusted_level_2"]].drop_duplicates()
    
ccs_icd10_map = ccs_icd10_map.rename(index=str,columns={"'MULTI CCS LVL 1'":"CCS_LVL_1_code",\
                                                            "'MULTI CCS LVL 1 LABEL'":"CCS_LVL_1_Dec",\
                                                            "'MULTI CCS LVL 2'":"CCS_LVL_2_code",\
                                                            "adjusted_level_2":"CCS_LVL_2_Dec"})

In [10]:
ccs_icd10_map.head(3).append(ccs_icd10_map.tail(3))


AttributeError: 'DataFrame' object has no attribute 'append'

In [11]:
ccs_icd10_map.shape

(72477, 6)

In [61]:
    # Claims Table Query
    claims_raw = wr.athena.read_sql_query("SELECT * from claims limit 1000",\
                                          database="client-reporting-diamond",\
                                          boto3_session=session,\
                                          workgroup='DataScientists', \
                                          ctas_approach=False) 

In [170]:
claims_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Columns: 163 entries, diagnosis_1_cd to icd10_3digit
dtypes: Int32(6), Int64(6), boolean(3), datetime64[ns](12), float64(16), object(21), string(99)
memory usage: 1.2+ MB


In [166]:
claims_raw.head(3)

,diagnosis_1_cd,diagnosis_2_cd,diagnosis_3_cd,diagnosis_4_cd,diagnosis_5_cd,diagnosis_6_cd,diagnosis_7_cd,diagnosis_8_cd,claim_pay_to_nm,sub_place_of_svc_cd,plan_code,network_provider_type_nm,seq_prov_contract,provider_par_stat,original_reference_number,claim_num_key,seq_group_id,provider_id,vendor_id,claim_location_country_cd,user_defined_2,user_defined_3,treatment_start_dt,claim_thru_date,days_supply,specialty_ind,rxp_match_key,rxp_paid_date,drug_brand_name,generic_brand_ind,ther_class_spc,rxp_billed_amt,rxp_paid_amt,pharmacy_name,spfc_thrpdc_clss,drug_treatment_major_group,drug_treatment_minor_group,spfc_thrpdc_desc,drug_treatment_indication,allowed_amt,billed_amt,bma_amt,bma_reason_cd,claim_status_txt,company_code,copayment_1_amt,copayment_2_amt,deductible_amt,gl_ref_cd,line_item_num_key,...,multi_ccs_lvl_1_label,multi_ccs_lvl_2,multi_ccs_lvl_2_label,hcc_flag,treatment_desc,cm_procedure_flag,cm_diagnosis_flag,cm_flag,case_type_nm,claim_location_country_nm,region,claim_type_nm,type_of_policy,segment,rating,par,plan_segment,is_aso_cd,msc,code,major,minor,category,source_bucket_path_nm,icd10,detail_svc_date_month,detail_svc_date_year,detail_svc_date_yr_mon,key,unplanned_days_since_last_ip_visit,unplanned_30d_readmission_flag,unplanned_60d_readmission_flag,unplanned_primary_admission,unplanned_ip_visit_level_incurred_date,unplanned_ip_visit_level_discharge_date,unplanned_los_ip_by_visit,key2,all_cause_days_since_last_ip_visit,all_cause_30d_readmission_flag,all_cause_60d_readmission_flag,all_cause_primary_admission,all_cause_ip_visit_level_incurred_date,all_cause_ip_visit_level_discharge_date,all_cause_los_ip_by_visit,cignalinks_prov_first_name,cignalinks_prov_last_name,cignalinks_prov_id,__index_level_0__,ICD10,icd10_3digit
0,F15.20,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Provider,21,CLSTNDB,54,11834082,N,<NA>,105384625,48747,7204982,8835297,USA,202008587291485,<NA>,2020-03-13,2020-03-13,<NA>,<NA>,<NA>,NaT,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,3250.00,4500.00,1250.00,BA063,P,CLIC,0.00,0.00,0.00,790,1,...,Mental Illness,5.12,Substance-related disorders [661],1.0,BEHAVIORAL HEALTH ACCOMODATIONS CHEM DEPENDENT,<NA>,<NA>,False,Medical,United States of America (the),Americas,IP,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,Outpatient,010X,Surgical,IP Room & Board,INSTITUTIONAL,intl-euro-uk-data-prod/raw/diamond_sw/,F15,3,2020,202003,101260251053846255.12,9999.0,0.0,0.0,yes,2020-03-13,2020-03-13,1.0,101260255.12,9999.0,0.0,0.0,yes,2020-03-13,2020-03-13,1.0,<NA>,<NA>,<NA>,0,F1520,F15
1,F15.20,F15.20,F11.20,F12.20,F13.20,<NA>,<NA>,<NA>,Provider,21,CHSTNDA,48,11681297,N,<NA>,106705446,11073,7349999,9032447,USA,202021887290833,<NA>,2020-07-22,2020-07-22,<NA>,<NA>,<NA>,NaT,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1425.00,3800.00,2375.00,BA063,P,CHLIC,0.00,0.00,0.00,790,1,...,Mental Illness,5.12,Substance-related disorders [661],1.0,BEHAVIORAL HEALTH ACCOMODATIONS CHEM DEPENDENT,<NA>,<NA>,False,Medical,United States of America (the),Americas,IP,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,Outpatient,010X,Surgical,IP Room & Board,INSTITUTIONAL,intl-euro-uk-data-prod/raw/diamond_sw/,F15,7,2020,202007,36922381067054465.12,20.0,1.0,1.0,no,2020-07-22,2020-07-22,1.0,36922385.12,20.0,1.0,1.0,no,2020-07-22,2020-07-22,1.0,<NA>,<NA>,<NA>,3,F1520,F15
2,F10.20,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Provider,21,GISTNDAC,43,12726454,N,<NA>,111321000,148406,7621266,9372005,USA,202121987290295,<NA>,2021-07-27,2021-07-27,<NA>,<NA>,<NA>,NaT,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,2397.00,3995.00,1598.00,BA340,P,CGIC,0.00,0.00,0.00,790,1,...,Mental Illness,5.11,Alcohol-related disorders [660],1.0,BEHAVIORAL HEALTH ACCOMODATIONS CHEM DEPENDENT,<NA>,<NA>,False,Medical,United States of America (the),Americas,IP,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,Outpatient,010X,Surgical,IP Room & Board,INSTITUTIONAL,intl-euro-uk-data-prod/raw/diamond_sw/,F10,7,2021,202107,111858741113210005.11,1.0,0.0,0.0,no,2021-07-27,2021-07-27,1.0,111858745.11,1.0,0.0,0.0,no,2021-07-27,2021-0

In [63]:
# map CCS with Claims 
    claims_raw['ICD10'] = claims_raw['diagnosis_1_cd'].str.replace('.','').str.strip()
    claims_raw['icd10_3digit'] = claims_raw['diagnosis_1_cd'].str.replace('.','').str.strip().str[0:3]
    claims_df = pd.merge(claims_raw, ccs_icd10_map, left_on='ICD10', right_on='icd_code',how='left',suffixes=('', '_1'))

In [64]:
claims_df.head(3)

,diagnosis_1_cd,diagnosis_2_cd,diagnosis_3_cd,diagnosis_4_cd,diagnosis_5_cd,diagnosis_6_cd,diagnosis_7_cd,diagnosis_8_cd,claim_pay_to_nm,sub_place_of_svc_cd,plan_code,network_provider_type_nm,seq_prov_contract,provider_par_stat,original_reference_number,claim_num_key,seq_group_id,provider_id,vendor_id,claim_location_country_cd,user_defined_2,user_defined_3,treatment_start_dt,claim_thru_date,days_supply,specialty_ind,rxp_match_key,rxp_paid_date,drug_brand_name,generic_brand_ind,ther_class_spc,rxp_billed_amt,rxp_paid_amt,pharmacy_name,spfc_thrpdc_clss,drug_treatment_major_group,drug_treatment_minor_group,spfc_thrpdc_desc,drug_treatment_indication,allowed_amt,billed_amt,bma_amt,bma_reason_cd,claim_status_txt,company_code,copayment_1_amt,copayment_2_amt,deductible_amt,gl_ref_cd,line_item_num_key,...,cm_diagnosis_flag,cm_flag,case_type_nm,claim_location_country_nm,region,claim_type_nm,type_of_policy,segment,rating,par,plan_segment,is_aso_cd,msc,code,major,minor,category,source_bucket_path_nm,icd10,detail_svc_date_month,detail_svc_date_year,detail_svc_date_yr_mon,key,unplanned_days_since_last_ip_visit,unplanned_30d_readmission_flag,unplanned_60d_readmission_flag,unplanned_primary_admission,unplanned_ip_visit_level_incurred_date,unplanned_ip_visit_level_discharge_date,unplanned_los_ip_by_visit,key2,all_cause_days_since_last_ip_visit,all_cause_30d_readmission_flag,all_cause_60d_readmission_flag,all_cause_primary_admission,all_cause_ip_visit_level_incurred_date,all_cause_ip_visit_level_discharge_date,all_cause_los_ip_by_visit,cignalinks_prov_first_name,cignalinks_prov_last_name,cignalinks_prov_id,__index_level_0__,ICD10,icd10_3digit,'ICD-10-CM CODE',icd_code,CCS_LVL_1_code,CCS_LVL_1_Dec,CCS_LVL_2_code,CCS_LVL_2_Dec
0,F15.20,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Provider,21,CLSTNDB,54,11834082,N,<NA>,105384625,48747,7204982,8835297,USA,202008587291485,<NA>,2020-03-13,2020-03-13,<NA>,<NA>,<NA>,NaT,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,3250.00,4500.00,1250.00,BA063,P,CLIC,0.00,0.00,0.00,790,1,...,<NA>,False,Medical,United States of America (the),Americas,IP,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,Outpatient,010X,Surgical,IP Room & Board,INSTITUTIONAL,intl-euro-uk-data-prod/raw/diamond_sw/,F15,3,2020,202003,101260251053846255.12,9999.0,0.0,0.0,yes,2020-03-13,2020-03-13,1.0,101260255.12,9999.0,0.0,0.0,yes,2020-03-13,2020-03-13,1.0,<NA>,<NA>,<NA>,0,F1520,F15,'F1520',F1520,'5',Mental Illness,'5.12',Substance-related disorders
1,F15.20,F15.20,F11.20,F12.20,F13.20,<NA>,<NA>,<NA>,Provider,21,CHSTNDA,48,11681297,N,<NA>,106705446,11073,7349999,9032447,USA,202021887290833,<NA>,2020-07-22,2020-07-22,<NA>,<NA>,<NA>,NaT,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1425.00,3800.00,2375.00,BA063,P,CHLIC,0.00,0.00,0.00,790,1,...,<NA>,False,Medical,United States of America (the),Americas,IP,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,Outpatient,010X,Surgical,IP Room & Board,INSTITUTIONAL,intl-euro-uk-data-prod/raw/diamond_sw/,F15,7,2020,202007,36922381067054465.12,20.0,1.0,1.0,no,2020-07-22,2020-07-22,1.0,36922385.12,20.0,1.0,1.0,no,2020-07-22,2020-07-22,1.0,<NA>,<NA>,<NA>,3,F1520,F15,'F1520',F1520,'5',Mental Illness,'5.12',Substance-related disorders
2,F10.20,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Provider,21,GISTNDAC,43,12726454,N,<NA>,111321000,148406,7621266,9372005,USA,202121987290295,<NA>,2021-07-27,2021-07-27,<NA>,<NA>,<NA>,NaT,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,2397.00,3995.00,1598.00,BA340,P,CGIC,0.00,0.00,0.00,790,1,...,<NA>,False,Medical,United States of America (the),Americas,IP,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,Outpatient,010X,Surgical,IP Room & Board,INSTITUTIONAL,intl-euro-uk-data-prod/raw/diamond_sw/,F10,7,2021,202107,111858741113210005.11,1.0,0.0,0.0,no,2021-07-27,2021-07-27,1.0,111858745.11,1.0,0.0,0.0,no,2021-07-27,2021-07-27,1.0,<NA>,<NA>,<NA>,4,F1020,F10,'F1020',F1020,'5',Mental Illness,'5.11',Alcohol-related disorders


In [65]:
claims_df.dtypes

diagnosis_1_cd                                     string
diagnosis_2_cd                                     string
diagnosis_3_cd                                     string
diagnosis_4_cd                                     string
diagnosis_5_cd                                     string
diagnosis_6_cd                                     string
diagnosis_7_cd                                     string
diagnosis_8_cd                                     string
claim_pay_to_nm                                    string
sub_place_of_svc_cd                                string
plan_code                                          string
network_provider_type_nm                           string
seq_prov_contract                                  object
provider_par_stat                                  string
original_reference_number                          string
claim_num_key                                      string
seq_group_id                                       object
provider_id   

In [66]:
claims_df.select_dtypes(include=['string']).head(3)

,diagnosis_1_cd,diagnosis_2_cd,diagnosis_3_cd,diagnosis_4_cd,diagnosis_5_cd,diagnosis_6_cd,diagnosis_7_cd,diagnosis_8_cd,claim_pay_to_nm,sub_place_of_svc_cd,plan_code,network_provider_type_nm,provider_par_stat,original_reference_number,claim_num_key,claim_location_country_cd,user_defined_2,user_defined_3,days_supply,specialty_ind,rxp_match_key,drug_brand_name,generic_brand_ind,ther_class_spc,rxp_billed_amt,rxp_paid_amt,pharmacy_name,spfc_thrpdc_clss,drug_treatment_major_group,drug_treatment_minor_group,spfc_thrpdc_desc,drug_treatment_indication,bma_reason_cd,claim_status_txt,company_code,gl_ref_cd,reason_for_rejection_txt,place_of_svc_cd,treatment_cd,treatment_modifier_cd,sub_line_cd,med_def_code,treatment_end_dt,is_prof_claim_type,post_date_yyyymm,group_id,group_id_6substr,group_nm,region__c,policy__c,book_of_business_nm,diagnosis_9_cd,nsa_logic,incurred_year_month,place_of_svc_code,place_of_svc_desc,pims_id,network_provider_id,prov_first_name,prov_last_name,network_provider_nm,network_provider_category_nm,diagnosis_1_desc,diagnosis_2_desc,diagnosis_3_desc,diagnosis_4_desc,diagnosis_5_desc,diagnosis_6_desc,diagnosis_7_desc,diagnosis_8_desc,diagnosis_9_desc,icd_10_cm_code,is_chronic_cd,ccsr_category_1_description,multi_ccs_lvl_1,multi_ccs_lvl_1_label,multi_ccs_lvl_2,multi_ccs_lvl_2_label,treatment_desc,case_type_nm,claim_location_country_nm,region,claim_type_nm,msc,code,major,minor,category,source_bucket_path_nm,icd10,key,unplanned_primary_admission,key2,all_cause_primary_admission,cignalinks_prov_first_name,cignalinks_prov_last_name,cignalinks_prov_id,icd10_3digit
0,F15.20,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Provider,21,CLSTNDB,54,N,<NA>,105384625,USA,202008587291485,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,BA063,P,CLIC,790,<NA>,<NA>,1002,<NA>,,9300,<NA>,N,202004,02433B004,02433B,KGML KINROSS GLOBAL MOBILITY LTD,Americas,<NA>,Americas,<NA>,No Match,2020-03,<NA>,<NA>,<NA>,7204982,<NA>,VOGUE RECOVERY CALIFORNIA,<NA>,Y,OTHER STIMULANT DEPENDEN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,F1520,C,Stimulant-related disorders,5,Mental Illness,5.12,Substance-related disorders [661],BEHAVIORAL HEALTH ACCOMODATIONS CHEM DEPENDENT,Medical,United States of America (the),Americas,IP,Outpatient,010X,Surgical,IP Room & Board,INSTITUTIONAL,intl-euro-uk-data-prod/raw/diamond_sw/,F15,101260251053846255.12,yes,101260255.12,yes,<NA>,<NA>,<NA>,F15
1,F15.20,F15.20,F11.20,F12.20,F13.20,<NA>,<NA>,<NA>,Provider,21,CHSTNDA,48,N,<NA>,106705446,USA,202021887290833,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,BA063,P,CHLIC,790,<NA>,<NA>,1002,<NA>,,9300,<NA>,N,202008,00234H999,00234H,PARSONS BENEFITS,Americas,<NA>,Americas,<NA>,No Match,2020-07,<NA>,<NA>,<NA>,7349999,<NA>,10 ACRE RANCH INC,<NA>,Y,OTHER STIMULANT DEPENDEN,OTHER STIMULANT DEPENDEN,"OPIOID DEPENDENCE,UNCOMP","CANNABIS DEPENDENCE,UNCO","SEDATIVE,HYPNOTIC OR ANX",<NA>,<NA>,<NA>,<NA>,F1520,C,Stimulant-related disorders,5,Mental Illness,5.12,Substance-related disorders [661],BEHAVIORAL HEALTH ACCOMODATIONS CHEM DEPENDENT,Medical,United States of America (the),Americas,IP,Outpatient,010X,Surgical,IP Room & Board,INSTITUTIONAL,intl-euro-uk-data-prod/raw/diamond_sw/,F15,36922381067054465.12,no,36922385.12,no,<NA>,<NA>,<NA>,F15
2,F10.20,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Provider,21,GISTNDAC,43,N,<NA>,111321000,USA,202121987290295,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,BA340,P,CGIC,790,<NA>,<NA>,1002,<NA>,,9200,<NA>,N,202109,08626A001,08626A,ST. GEORGE'S UNIVERSITY,Americas,<NA>,Americas,<NA>,"{'bma_reason': 'CABMA', 'user_defined_1': 'SVC...",2021-07,<NA>,<NA>,<NA>,7621266,<NA>,MALLARD LAKE RANCH DETOX,<NA>,Y,"ALCOHOL DEPENDENCE,UNCOM",<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,F1020,C,Alcohol-related disorders,5,Mental Illness,5.11,Alcohol-related disorders [660],BEHAVIORAL HEALTH ACCOMODATIONS CHEM DEPENDENT,Medical,United States of America (the),Americas,IP,Outpatient,010X,Surgical,IP Room & Board,INSTITU

In [67]:
# define rolling 12 months period
    claims_df['max_incurred_year_month'] = max(claims_df['incurred_year_month'])
    claims_df['max_incurred_year_month_dt'] = pd.to_datetime(claims_df['max_incurred_year_month'], format='%Y-%m')  
    
    

In [68]:
    claims_df.dtypes

diagnosis_1_cd                                     string
diagnosis_2_cd                                     string
diagnosis_3_cd                                     string
diagnosis_4_cd                                     string
diagnosis_5_cd                                     string
diagnosis_6_cd                                     string
diagnosis_7_cd                                     string
diagnosis_8_cd                                     string
claim_pay_to_nm                                    string
sub_place_of_svc_cd                                string
plan_code                                          string
network_provider_type_nm                           string
seq_prov_contract                                  object
provider_par_stat                                  string
original_reference_number                          string
claim_num_key                                      string
seq_group_id                                       object
provider_id   

In [69]:
    claims_df.select_dtypes(include=['datetime']).head(3)

,treatment_start_dt,claim_thru_date,rxp_paid_date,detail_svc_date,claim_paid_dt,check_date,insert_datetime,post_date_imputed,unplanned_ip_visit_level_incurred_date,unplanned_ip_visit_level_discharge_date,all_cause_ip_visit_level_incurred_date,all_cause_ip_visit_level_discharge_date,max_incurred_year_month_dt
0,2020-03-13,2020-03-13,NaT,2020-03-13,2020-04-15 01:49:33,2020-04-15 02:37:25,2020-04-10 11:32:23,2020-04-15 01:49:33,2020-03-13,2020-03-13,2020-03-13,2020-03-13,2023-08-01
1,2020-07-22,2020-07-22,NaT,2020-07-22,2020-08-29 02:16:00,2020-08-29 03:03:09,2020-08-26 11:57:18,2020-08-29 02:16:00,2020-07-22,2020-07-22,2020-07-22,2020-07-22,2023-08-01
2,2021-07-27,2021-07-27,NaT,2021-07-27,2021-09-04 02:24:44,2021-09-04 03:28:50,2021-09-01 11:47:26,2021-09-04 02:24:44,2021-07-27,2021-07-27,2021-07-27,2021-07-27,2023-08-01


In [70]:
    max_date = claims_df['max_incurred_year_month'].unique().tolist()    #converting to list

In [71]:
    max_date

['2023-08']

In [72]:
#-dateoffset by one year. Max -1 year
    #month offset -1, plus month end(EOM)
    
    from pandas.tseries.offsets import MonthEnd
    claims_df['rolling_12mth_start'] = (claims_df['max_incurred_year_month_dt'] - pd.DateOffset(years=1))
    claims_df['rolling_12mth_end'] = (claims_df['max_incurred_year_month_dt'] - pd.DateOffset(months=1)) + MonthEnd(1)
    
    claims_df['detail_svc_date'] = pd.to_datetime(claims_df['detail_svc_date'], infer_datetime_format=True)
    
    claims_df['rolling_12mth_claims'] = np.where((claims_df['detail_svc_date']>=claims_df['rolling_12mth_start']) & \
                                (claims_df['detail_svc_date']<=claims_df['rolling_12mth_end']), "rolling 12 months", "N")

In [73]:
    claims_df[['rolling_12mth_start','rolling_12mth_end','max_incurred_year_month_dt','detail_svc_date','rolling_12mth_claims']].head(10)

,rolling_12mth_start,rolling_12mth_end,max_incurred_year_month_dt,detail_svc_date,rolling_12mth_claims
0,2022-08-01,2023-07-31,2023-08-01,2020-03-13,N
1,2022-08-01,2023-07-31,2023-08-01,2020-07-22,N
2,2022-08-01,2023-07-31,2023-08-01,2021-07-27,N
3,2022-08-01,2023-07-31,2023-08-01,2021-05-01,N
4,2022-08-01,2023-07-31,2023-08-01,2020-08-31,N
5,2022-08-01,2023-07-31,2023-08-01,2020-03-16,N
6,2022-08-01,2023-07-31,2023-08-01,2021-08-20,N
7,2022-08-01,2023-07-31,2023-08-01,2019-12-11,N
8,2022-08-01,2023-07-31,2023-08-01,2023-07-10,rolling 12 months
9,2022-08-01,2023-07-31,2023-08-01,2022-10-01,rolling 12 months


In [74]:
# C=Chronic A=Acute B=Both and N=Not Applicable
    claims_df['cci_chronic'] = np.where(claims_df['is_chronic_cd'].isin(['C', 'B']),"chronic condition", 
                                        "non-chronic condition")

In [75]:
    claims_df[['cci_chronic','is_chronic_cd']].tail(10)

,cci_chronic,is_chronic_cd
990,chronic condition,C
991,chronic condition,C
992,chronic condition,C
993,non-chronic condition,A
994,chronic condition,C
995,chronic condition,C
996,chronic condition,C
997,chronic condition,C
998,chronic condition,C
999,chronic condition,C


In [76]:
# create "visit key" in claims
    claims_df['visit_key'] = claims_df['unique_member_id'].astype(str) + ' ' + claims_df['detail_svc_date'].astype(str)

In [77]:
    claims_df[['cci_chronic','is_chronic_cd','visit_key','unique_member_id','detail_svc_date']].tail()

,cci_chronic,is_chronic_cd,visit_key,unique_member_id,detail_svc_date
995,chronic condition,C,10696136 2021-02-04,10696136,2021-02-04
996,chronic condition,C,9564055 2019-02-28,9564055,2019-02-28
997,chronic condition,C,7455022 2020-08-03,7455022,2020-08-03
998,chronic condition,C,11812200 2022-01-17,11812200,2022-01-17
999,chronic condition,C,1845394 2021-09-15,1845394,2021-09-15


In [78]:
# to define "at risk", need to get the visit count for each chronic condition
    # at risk: 1,2 or 3 chronic and max(visit count for each chronic)==1
    # low chronic: 1 or 2 chronic and max(visit count) >=2
    # high chronic: 3 or more chronic and max(visit count) >=2

    # max visit when chronic defined by icd9 full description
    # new df full with claims_df group by and new column aggreate icd_10_full_visit_cnt
    
    member_visit_state_icd10_full = claims_df.loc[claims_df['cci_chronic'] == 'chronic condition'].groupby(
            ['unique_member_id','incurred_year','book_of_business_nm','diagnosis_1_cd']).agg(
            icd10_full_visit_cnt = ('visit_key', 'nunique')).reset_index()

In [79]:
    member_visit_state_icd10_full.tail(3)

,unique_member_id,incurred_year,book_of_business_nm,diagnosis_1_cd,icd10_full_visit_cnt
143,12579003,2023,Americas,F50.2,5
144,12771202,2023,Americas,F11.20,1
145,12898221,2023,Americas,F10.20,13


In [80]:
#member_visit_state_icd10_full[member_visit_state_icd10_full.unique_member_id==9564055]

In [81]:
# new df max from full and group. and icd_10_visit_cnt from above.transform  ==full[icd10_full_visit_cnt]   
    
    member_visit_state_icd10_full_max = member_visit_state_icd10_full[member_visit_state_icd10_full.groupby(['unique_member_id',
            'incurred_year','book_of_business_nm'])['icd10_full_visit_cnt'].transform(max) == member_visit_state_icd10_full['icd10_full_visit_cnt']]

    member_visit_state_icd10_full_max.drop('diagnosis_1_cd', axis=1, inplace=True)
    member_visit_state_icd10_full_max.drop_duplicates(inplace = True)

In [82]:
member_visit_state_icd10_full_max[member_visit_state_icd10_full_max.unique_member_id==9564055]

,unique_member_id,incurred_year,book_of_business_nm,icd10_full_visit_cnt
57,9564055,2018,Americas,8
59,9564055,2019,Americas,6


In [83]:
# max visit when chronic defined by icd9 3 digits
    member_visit_state_icd10_3digit = claims_df.loc[claims_df['cci_chronic'] == 'chronic condition'].groupby(
            ['unique_member_id','incurred_year','book_of_business_nm','icd10_3digit']).agg(
            icd10_3digit_visit_cnt = ('visit_key', 'nunique')).reset_index()
    member_visit_state_icd10_3digit_max = member_visit_state_icd10_3digit[member_visit_state_icd10_3digit.groupby(
            ['unique_member_id','incurred_year','book_of_business_nm'])['icd10_3digit_visit_cnt'].transform(max) == member_visit_state_icd10_3digit['icd10_3digit_visit_cnt']]

    member_visit_state_icd10_3digit_max.drop('icd10_3digit', axis=1, inplace=True)
    member_visit_state_icd10_3digit_max.drop_duplicates(inplace = True)

In [84]:
    member_visit_state_icd10_3digit_max.head(3)

,unique_member_id,incurred_year,book_of_business_nm,icd10_3digit_visit_cnt
0,368582,2019,Americas,1
1,777070,2018,Americas,2
2,966183,2021,Americas,9


In [85]:
# max visit when chronic defined by ccs level 2
    member_visit_state_ccs_lv2 = claims_df.loc[claims_df['cci_chronic'] == 'chronic condition'].groupby(
            ['unique_member_id','incurred_year','book_of_business_nm','CCS_LVL_2_code']).agg(
            ccs_lv2_visit_cnt = ('visit_key', 'nunique')).reset_index()

    member_visit_state_ccs_lv2_max = member_visit_state_ccs_lv2[member_visit_state_ccs_lv2.groupby(
            ['unique_member_id','incurred_year','book_of_business_nm'])['ccs_lv2_visit_cnt'].transform(max) == member_visit_state_ccs_lv2['ccs_lv2_visit_cnt']]

    member_visit_state_ccs_lv2_max.drop('CCS_LVL_2_code', axis=1, inplace=True)
    member_visit_state_ccs_lv2_max.drop_duplicates(inplace = True)

In [86]:
    member_visit_state_ccs_lv2_max.head(3)

,unique_member_id,incurred_year,book_of_business_nm,ccs_lv2_visit_cnt
0,368582,2019,Americas,1
1,777070,2018,Americas,2
2,966183,2021,Americas,9


In [87]:
 # max visit when chronic defined by ccs level 1
    member_visit_state_ccs_lv1 = claims_df.loc[claims_df['cci_chronic'] == 'chronic condition'].groupby(
            ['unique_member_id','incurred_year','book_of_business_nm','CCS_LVL_1_code']).agg(
            ccs_lv1_visit_cnt = ('visit_key', 'nunique')).reset_index()

    member_visit_state_ccs_lv1_max = member_visit_state_ccs_lv1[member_visit_state_ccs_lv1.groupby(
            ['unique_member_id','incurred_year','book_of_business_nm'])['ccs_lv1_visit_cnt'].transform(max) == member_visit_state_ccs_lv1['ccs_lv1_visit_cnt']]

    member_visit_state_ccs_lv1_max.drop('CCS_LVL_1_code', axis=1, inplace=True)
    member_visit_state_ccs_lv1_max.drop_duplicates(inplace = True)

In [88]:
member_visit_state_ccs_lv1_max.head(3)

,unique_member_id,incurred_year,book_of_business_nm,ccs_lv1_visit_cnt
0,368582,2019,Americas,1
1,777070,2018,Americas,2
2,966183,2021,Americas,9


In [89]:
# merge visit cnt tables into one
    member_visit_state_0 = member_visit_state_icd10_full_max.merge(member_visit_state_icd10_3digit_max, how='left',
                                                                 on=['unique_member_id','incurred_year','book_of_business_nm'])
    member_visit_state_1 = member_visit_state_0.merge(member_visit_state_ccs_lv2_max, how='left',
                                                                 on=['unique_member_id','incurred_year','book_of_business_nm'])
    member_visit_state = member_visit_state_1.merge(member_visit_state_ccs_lv1_max, how='left',
                                                                 on=['unique_member_id','incurred_year','book_of_business_nm'])
    member_visit_state.head()

,unique_member_id,incurred_year,book_of_business_nm,icd10_full_visit_cnt,icd10_3digit_visit_cnt,ccs_lv2_visit_cnt,ccs_lv1_visit_cnt
0,368582,2019,Americas,1,1,1,1
1,777070,2018,Americas,2,2,2,2
2,966183,2021,Americas,9,9,9,9
3,1357084,2023,Americas,15,15,15,15
4,1648431,2019,Americas,1,1,1,1


# max visit when chronic defined by icd9 full description

In [90]:
#------------- visit cnt table for rolling 12 months -------------#

    # max visit when chronic defined by icd9 full description
    member_visit_state_icd10_full_12mth = claims_df.loc[(claims_df['cci_chronic'] == 'chronic condition')&\
                                                        (claims_df['rolling_12mth_claims']=='rolling 12 months')].groupby(
            ['unique_member_id','rolling_12mth_claims','book_of_business_nm','diagnosis_1_cd']).agg(
            icd10_full_visit_cnt = ('visit_key', 'nunique')).reset_index()

    member_visit_state_icd10_full_max_12mth = member_visit_state_icd10_full_12mth[member_visit_state_icd10_full_12mth.groupby(['unique_member_id',
            'rolling_12mth_claims','book_of_business_nm'])['icd10_full_visit_cnt'].transform(max) == member_visit_state_icd10_full_12mth['icd10_full_visit_cnt']]

    member_visit_state_icd10_full_max_12mth.drop('diagnosis_1_cd', axis=1, inplace=True)
    member_visit_state_icd10_full_max_12mth.drop_duplicates(inplace = True)




In [91]:
    member_visit_state_icd10_full_max_12mth.head()

,unique_member_id,rolling_12mth_claims,book_of_business_nm,icd10_full_visit_cnt
0,1357084,rolling 12 months,Americas,15
1,2891819,rolling 12 months,Americas,1
2,3792853,rolling 12 months,Americas,2
3,5611369,rolling 12 months,Americas,5
4,6169422,rolling 12 months,Americas,1


# max visit when chronic defined by icd9 3 digits

In [92]:
 # max visit when chronic defined by icd9 3 digits
    member_visit_state_icd10_3digit_12mth = claims_df.loc[(claims_df['cci_chronic'] == 'chronic condition')&(claims_df['rolling_12mth_claims']=='rolling 12 months')].groupby(
            ['unique_member_id','rolling_12mth_claims','book_of_business_nm','icd10_3digit']).agg(
            icd10_3digit_visit_cnt = ('visit_key', 'nunique')).reset_index()
    member_visit_state_icd10_3digit_max_12mth = member_visit_state_icd10_3digit_12mth[member_visit_state_icd10_3digit_12mth.groupby(
            ['unique_member_id','rolling_12mth_claims','book_of_business_nm'])['icd10_3digit_visit_cnt'].transform(max) == member_visit_state_icd10_3digit_12mth['icd10_3digit_visit_cnt']]

    member_visit_state_icd10_3digit_max_12mth.drop('icd10_3digit', axis=1, inplace=True)
    member_visit_state_icd10_3digit_max_12mth.drop_duplicates(inplace = True)
    member_visit_state_icd10_3digit_max_12mth.head()

,unique_member_id,rolling_12mth_claims,book_of_business_nm,icd10_3digit_visit_cnt
0,1357084,rolling 12 months,Americas,15
1,2891819,rolling 12 months,Americas,1
2,3792853,rolling 12 months,Americas,2
3,5611369,rolling 12 months,Americas,5
4,6169422,rolling 12 months,Americas,1


# max visit when chronic defined by ccs level 2

In [93]:
 # max visit when chronic defined by ccs level 2
    member_visit_state_ccs_lv2_12mth = claims_df.loc[(claims_df['cci_chronic'] == 'chronic condition')&(claims_df['rolling_12mth_claims']=='rolling 12 months')].groupby(
            ['unique_member_id','rolling_12mth_claims','book_of_business_nm','CCS_LVL_2_code']).agg(
            ccs_lv2_visit_cnt = ('visit_key', 'nunique')).reset_index()

    member_visit_state_ccs_lv2_max_12mth = member_visit_state_ccs_lv2_12mth[member_visit_state_ccs_lv2_12mth.groupby(
            ['unique_member_id','rolling_12mth_claims','book_of_business_nm'])['ccs_lv2_visit_cnt'].transform(max) == member_visit_state_ccs_lv2_12mth['ccs_lv2_visit_cnt']]

    member_visit_state_ccs_lv2_max_12mth.drop('CCS_LVL_2_code', axis=1, inplace=True)
    member_visit_state_ccs_lv2_max_12mth.drop_duplicates(inplace = True)
    member_visit_state_ccs_lv2_max_12mth.head()

,unique_member_id,rolling_12mth_claims,book_of_business_nm,ccs_lv2_visit_cnt
0,1357084,rolling 12 months,Americas,15
1,2891819,rolling 12 months,Americas,1
2,3792853,rolling 12 months,Americas,2
3,5611369,rolling 12 months,Americas,5
4,6169422,rolling 12 months,Americas,1


# max visit when chronic defined by ccs level 1

In [94]:
# max visit when chronic defined by ccs level 1
    member_visit_state_ccs_lv1_12mth = claims_df.loc[(claims_df['cci_chronic'] == 'chronic condition')&(claims_df['rolling_12mth_claims']=='rolling 12 months')].groupby(
            ['unique_member_id','rolling_12mth_claims','book_of_business_nm','CCS_LVL_1_code']).agg(
            ccs_lv1_visit_cnt = ('visit_key', 'nunique')).reset_index()

    member_visit_state_ccs_lv1_max_12mth = member_visit_state_ccs_lv1_12mth[member_visit_state_ccs_lv1_12mth.groupby(
            ['unique_member_id','rolling_12mth_claims','book_of_business_nm'])['ccs_lv1_visit_cnt'].transform(max) == member_visit_state_ccs_lv1_12mth['ccs_lv1_visit_cnt']]

    member_visit_state_ccs_lv1_max_12mth.drop('CCS_LVL_1_code', axis=1, inplace=True)
    member_visit_state_ccs_lv1_max_12mth.drop_duplicates(inplace = True)
    member_visit_state_ccs_lv1_max_12mth.head()

,unique_member_id,rolling_12mth_claims,book_of_business_nm,ccs_lv1_visit_cnt
0,1357084,rolling 12 months,Americas,15
1,2891819,rolling 12 months,Americas,1
2,3792853,rolling 12 months,Americas,2
3,5611369,rolling 12 months,Americas,5
4,6169422,rolling 12 months,Americas,1


# merge visit cnt tables into one

In [95]:
# merge visit cnt tables into one
    member_visit_state_0_12mth = member_visit_state_icd10_full_max_12mth.merge(member_visit_state_icd10_3digit_max_12mth, how='left',
                                                                 on=['unique_member_id','rolling_12mth_claims','book_of_business_nm'])
    member_visit_state_1_12mth = member_visit_state_0_12mth.merge(member_visit_state_ccs_lv2_max_12mth, how='left',
                                                                 on=['unique_member_id','rolling_12mth_claims','book_of_business_nm'])
    member_visit_state_12mth = member_visit_state_1_12mth.merge(member_visit_state_ccs_lv1_max_12mth, how='left',
                                                                 on=['unique_member_id','rolling_12mth_claims','book_of_business_nm'])
    member_visit_state_12mth.head()

,unique_member_id,rolling_12mth_claims,book_of_business_nm,icd10_full_visit_cnt,icd10_3digit_visit_cnt,ccs_lv2_visit_cnt,ccs_lv1_visit_cnt
0,1357084,rolling 12 months,Americas,15,15,15,15
1,2891819,rolling 12 months,Americas,1,1,1,1
2,3792853,rolling 12 months,Americas,2,2,2,2
3,5611369,rolling 12 months,Americas,5,5,5,5
4,6169422,rolling 12 months,Americas,1,1,1,1


In [96]:
    member_visit_state_12mth = member_visit_state_12mth.rename(index=str,columns={'rolling_12mth_claims':'incurred_year'})
    member_visit_state_all = member_visit_state.append(member_visit_state_12mth,ignore_index=True)

In [97]:
    member_visit_state_all.head(3)

,unique_member_id,incurred_year,book_of_business_nm,icd10_full_visit_cnt,icd10_3digit_visit_cnt,ccs_lv2_visit_cnt,ccs_lv1_visit_cnt
0,368582,2019,Americas,1,1,1,1
1,777070,2018,Americas,2,2,2,2
2,966183,2021,Americas,9,9,9,9


#  Health State Flag logic Implementation

In [98]:
# Create Chronic Condition pivot table, based on chronic claims
    member_chronic_state = claims_df.loc[claims_df['cci_chronic'] == 'chronic condition'].groupby(['unique_member_id','incurred_year','book_of_business_nm']).agg(
                                                icd10_full_nunique = ('diagnosis_1_cd', 'nunique'),
                                                icd10_3digit_nunique = ('icd10_3digit','nunique'),
                                                ccs_lv2_nunique = ('CCS_LVL_2_code','nunique'),
                                                ccs_lv1_nunique = ('CCS_LVL_1_code', 'nunique')).reset_index()

In [99]:
member_chronic_state.head(3)

,unique_member_id,incurred_year,book_of_business_nm,icd10_full_nunique,icd10_3digit_nunique,ccs_lv2_nunique,ccs_lv1_nunique
0,368582,2019,Americas,1,1,1,1
1,777070,2018,Americas,1,1,1,1
2,966183,2021,Americas,1,1,1,1


In [100]:
    member_chronic_state_12mth = claims_df.loc[(claims_df['cci_chronic'] == 'chronic condition')&(claims_df['rolling_12mth_claims']=='rolling 12 months')].groupby(['unique_member_id','rolling_12mth_claims','book_of_business_nm']).agg(
                                                icd10_full_nunique = ('diagnosis_1_cd', 'nunique'),
                                                icd10_3digit_nunique = ('icd10_3digit','nunique'),
                                                ccs_lv2_nunique = ('CCS_LVL_2_code','nunique'),
                                                ccs_lv1_nunique = ('CCS_LVL_1_code', 'nunique')).reset_index()

In [101]:
    member_chronic_state_12mth.head(3)

,unique_member_id,rolling_12mth_claims,book_of_business_nm,icd10_full_nunique,icd10_3digit_nunique,ccs_lv2_nunique,ccs_lv1_nunique
0,1357084,rolling 12 months,Americas,1,1,1,1
1,2891819,rolling 12 months,Americas,1,1,1,1
2,3792853,rolling 12 months,Americas,1,1,1,1


In [102]:
    member_chronic_state_12mth = member_chronic_state_12mth.rename(index=str,columns={'rolling_12mth_claims':'incurred_year'})
    member_chronic_state_12mth.head(3)
 

,unique_member_id,incurred_year,book_of_business_nm,icd10_full_nunique,icd10_3digit_nunique,ccs_lv2_nunique,ccs_lv1_nunique
0,1357084,rolling 12 months,Americas,1,1,1,1
1,2891819,rolling 12 months,Americas,1,1,1,1
2,3792853,rolling 12 months,Americas,1,1,1,1


In [103]:
   member_chronic_state_all = member_chronic_state.append(member_chronic_state_12mth,ignore_index=True)   


In [104]:
    member_chronic_state_all.head(3)

,unique_member_id,incurred_year,book_of_business_nm,icd10_full_nunique,icd10_3digit_nunique,ccs_lv2_nunique,ccs_lv1_nunique
0,368582,2019,Americas,1,1,1,1
1,777070,2018,Americas,1,1,1,1
2,966183,2021,Americas,1,1,1,1


# Create general health state pivot table
    

In [105]:
# Create general health state pivot table
    member_health_state = claims_df.groupby(['unique_member_id','book_of_business_nm','incurred_year']).agg(
                                        total_gross_amt_usd = ('paid_net_amt', 'sum'),
                                        claim_id_nunique = ('claim_num_key','nunique')).reset_index()

In [106]:
member_health_state.head(3)

,unique_member_id,book_of_business_nm,incurred_year,total_gross_amt_usd,claim_id_nunique
0,368582,Americas,2019,85.12,1
1,515886,Americas,2019,0.00,1
2,777070,Americas,2018,0.00,2


In [107]:
    member_health_state_12mth = claims_df[claims_df['rolling_12mth_claims']=='rolling 12 months'].groupby(['unique_member_id','rolling_12mth_claims','book_of_business_nm']).agg(
                                        total_gross_amt_usd = ('paid_net_amt', 'sum'),
                                        claim_id_nunique = ('claim_num_key','nunique')).reset_index()

    member_health_state_12mth = member_health_state_12mth.rename(index=str,columns={'rolling_12mth_claims':'incurred_year'})

    member_health_state_12mth.head(3)

,unique_member_id,incurred_year,book_of_business_nm,total_gross_amt_usd,claim_id_nunique
0,1357084,rolling 12 months,Americas,28735.63,17
1,2887918,rolling 12 months,Americas,890.42,1
2,2891819,rolling 12 months,Americas,1565.00,1


In [108]:
    member_health_state_all = member_health_state.append(member_health_state_12mth,ignore_index=True)
    

In [109]:
    member_health_state_all.head(3)

,unique_member_id,book_of_business_nm,incurred_year,total_gross_amt_usd,claim_id_nunique
0,368582,Americas,2019,85.12,1
1,515886,Americas,2019,0.00,1
2,777070,Americas,2018,0.00,2


# Merge two tables together to summarize health states

In [110]:
    # Merge two tables together to summarize health states
    member_health_state = member_health_state_all.merge(member_chronic_state_all, how='left', on=['unique_member_id','incurred_year','book_of_business_nm'])
    member_health_state = member_health_state.merge(member_visit_state_all, how='left', on=['unique_member_id','incurred_year','book_of_business_nm'])
    # Fill NA with zeros
    member_health_state.fillna(0, inplace=True)
    member_health_state.head()

,unique_member_id,book_of_business_nm,incurred_year,total_gross_amt_usd,claim_id_nunique,icd10_full_nunique,icd10_3digit_nunique,ccs_lv2_nunique,ccs_lv1_nunique,icd10_full_visit_cnt,icd10_3digit_visit_cnt,ccs_lv2_visit_cnt,ccs_lv1_visit_cnt
0,368582,Americas,2019,85.12,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,515886,Americas,2019,0.00,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,777070,Americas,2018,0.00,2,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0
3,966183,Americas,2021,4708.06,9,1.0,1.0,1.0,1.0,9.0,9.0,9.0,9.0
4,1356765,Americas,2017,0.00,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [111]:
    print('Assign the corresponding health state flag to each members.')

Assign the corresponding health state flag to each members.


# # Create Health State Flags for Healthy, Healthy with Claims, Low Chronic, High Chronic

In [112]:
def health_state_flag_fn(hs_table,hs_col,hs_visit_cnt):
        conditions = [
            (hs_table[hs_col] == 0),
            (hs_table[hs_col] != 0) & (hs_table[hs_visit_cnt]==1),
            (hs_table[hs_col] >= 1) & (hs_table[hs_col] <= 2) & (hs_table[hs_visit_cnt]>=2),
            (hs_table[hs_col] >= 3) & (hs_table[hs_visit_cnt]>=2)
        ]

        labels = [
            'Healthy',
            'At Risk',
            'Low Chronic',
            'High Chronic'
        ]

        newcol = np.select(conditions, labels, default=np.nan)
        return newcol

In [113]:
    member_health_state['health_state_icd10_3digit'] = health_state_flag_fn(member_health_state, 'icd10_3digit_nunique', 'icd10_3digit_visit_cnt')
    member_health_state['health_state_icd10_full'] = health_state_flag_fn(member_health_state, 'icd10_full_nunique', 'icd10_full_visit_cnt')
    member_health_state['health_state_ccslv2'] = health_state_flag_fn(member_health_state, 'ccs_lv2_nunique','ccs_lv2_visit_cnt')
    member_health_state['health_state_ccslv1'] = health_state_flag_fn(member_health_state, 'ccs_lv1_nunique','ccs_lv1_visit_cnt')

#  Segment Healthy with Claims from Healthy using claim threshold

In [115]:
    claim_threshold = 500



    member_health_state['claim_threshold_usd'] = claim_threshold

    hs_cols = ['health_state_icd10_3digit','health_state_icd10_full','health_state_ccslv2','health_state_ccslv1']

    for i in hs_cols:
        member_health_state[i] = np.where((member_health_state[i] == 'Healthy') & (member_health_state['total_gross_amt_usd'] >= claim_threshold),'Healthy with Claims', member_health_state[i])
       
    
    

#   Select which type of Health State aggregation is used - ICD9, CCS Level 2, CCS Level 1

In [116]:
    selected_hs_type = 'health_state_icd10_full'
    member_health_state['selected_hs_type'] = selected_hs_type
    member_health_state['health_state'] = member_health_state[selected_hs_type]

# import Eligibility table

In [119]:
# Eligibility Table Query
    elig_df = wr.athena.read_sql_query(sql="SELECT distinct unique_member_id_key,\
                                            elig_year_month,\
                                            book_of_business_nm,\
                                            subscriber_id,\
                                            person_number_id,\
                                            seq_group_id\
                                        FROM membership",
                                      database="client-reporting-diamond",
                                      boto3_session=session,
                                      workgroup='DataScientists',
                                      ctas_approach=False)

    print('Imported Eligibility Table')
    print(elig_df.shape)

Imported Eligibility Table
(31942660, 6)


In [ ]:
#  Add a field for eligible_yr by extracting the year information from eligible_month



In [120]:
    elig_df['eligible_yr'] = elig_df['elig_year_month'].dt.year
    elig_df['eligible_yr'] = elig_df['eligible_yr'].fillna(0)

In [121]:
elig_df.head(3)

,unique_member_id_key,elig_year_month,book_of_business_nm,subscriber_id,person_number_id,seq_group_id,eligible_yr
0,5626900,2018-01-01,Americas,204575850,04,60035,2018
1,5627527,2018-01-01,Americas,204580106,02,67226,2018
2,5630202,2018-01-01,Americas,204573410,01,31570,2018


In [122]:
# type conversion from float64 to int64
    elig_df['eligible_yr'] = [int(i) for i in elig_df['eligible_yr']]
    
    elig_df['member_id'] = elig_df['subscriber_id'] + elig_df['person_number_id']
    
    elig_df['max_elig_year_month'] = max_date[0]
    elig_df['max_elig_year_month_dt'] = pd.to_datetime(elig_df['max_elig_year_month'], format='%Y-%m')

In [124]:
#ROLLING 1 YEAR

    from pandas.tseries.offsets import MonthEnd
    elig_df['rolling_12mth_start'] = (elig_df['max_elig_year_month_dt'] - pd.DateOffset(years=1))
    elig_df['rolling_12mth_end'] = (elig_df['max_elig_year_month_dt'] - pd.DateOffset(months=1)) + MonthEnd(1)
    
    elig_df['elig_year_month'] = pd.to_datetime(elig_df['elig_year_month'], infer_datetime_format=True)
    
    elig_df['rolling_12mth'] = np.where((elig_df['elig_year_month']>=elig_df['rolling_12mth_start']) & \
                                (elig_df['elig_year_month']<=elig_df['rolling_12mth_end']), "rolling 12 months", "N")

In [125]:
    elig_df_1 = elig_df[['unique_member_id_key','member_id', 'book_of_business_nm', 'eligible_yr']].drop_duplicates()

In [126]:
    elig_df_12mth = elig_df[elig_df['rolling_12mth']=='rolling 12 months'][['unique_member_id_key','member_id', 'book_of_business_nm', 
                                                                         'rolling_12mth']].drop_duplicates()

In [127]:
    elig_df_12mth = elig_df_12mth.rename(index=str,columns={'rolling_12mth':'eligible_yr'})

In [128]:
    elig_df_1['eligible_yr'] = elig_df_1['eligible_yr'].apply(str)
    elig_df_all = elig_df_1.append(elig_df_12mth,ignore_index=True)

In [137]:
    elig_df_all.head(3)

,unique_member_id_key,member_id,book_of_business_nm,eligible_yr
0,5626900,20457585004,Americas,2018
1,5627527,20458010602,Americas,2018
2,5630202,20457341001,Americas,2018


In [135]:
    # Converting to string
    
    member_health_state['incurred_year'] = member_health_state['incurred_year'].apply(str)
    elig_df_all['eligible_yr'] = elig_df_all['eligible_yr'].apply(str)
    member_health_state['unique_member_id'] = member_health_state['unique_member_id'].apply(str)
    elig_df_all['book_of_business_nm'] = elig_df_all['book_of_business_nm'].apply(str)
    elig_df_all['unique_member_id_key'] = elig_df_all['unique_member_id_key'].apply(str)
    member_health_state['unique_member_id'] = member_health_state['unique_member_id'].apply(str)

# health state is added to the eligible table

In [138]:
# Left join the eligibility table to the member health state pivot tabl
    member_health_state['incurred_year'] = member_health_state['incurred_year'].apply(str)
    final_df_output = elig_df_all.merge(member_health_state, how='left', left_on=['unique_member_id_key','eligible_yr',
                                                                              'book_of_business_nm'], 
                                right_on=['unique_member_id','incurred_year','book_of_business_nm'],
                                suffixes=('_elig', '_member'))

In [139]:
    print('Left join eligibility table to member_health_state')
    print('Complete table joining to get the health state flag.')
    print(final_df_output.shape)

Left join eligibility table to member_health_state
Complete table joining to get the health state flag.
(3181882, 23)


In [140]:
  print('Transform the output dataset.')

Transform the output dataset.


In [141]:
# Get all the fields from Athena eligible table for tableau + Health state + eligible year + market
    
    field_list = ['unique_member_id_key', 'member_id','eligible_yr','book_of_business_nm','health_state_icd10_3digit',
                    'health_state_icd10_full','health_state_ccslv2','health_state_ccslv1','selected_hs_type', 'health_state']
    final_df_temp = pd.DataFrame()
    final_df = pd.DataFrame()
    for i in field_list:
        final_df[i] = final_df_output[i]

In [143]:
    final_df.head(3)

,unique_member_id_key,member_id,eligible_yr,book_of_business_nm,health_state_icd10_3digit,health_state_icd10_full,health_state_ccslv2,health_state_ccslv1,selected_hs_type,health_state
0,5626900,20457585004,2018,Americas,NaN,NaN,NaN,NaN,NaN,NaN
1,5627527,20458010602,2018,Americas,NaN,NaN,NaN,NaN,NaN,NaN
2,5630202,20457341001,2018,Americas,NaN,NaN,NaN,NaN,NaN,NaN


In [144]:
# Finalizing the health state of each eligible members
    # Label "Non-Utilizer" to members with no claim
    health_state_list = ['health_state_icd10_3digit','health_state_icd10_full','health_state_ccslv2',
                         'health_state_ccslv1', 'health_state']    

In [161]:
    final_df['selected_hs_type'] = selected_hs_type
    for i in health_state_list:
        final_df[i].fillna('Non-Utilizer',inplace=True)
        final_df[i] = np.where(final_df[i] == 'Healthy','Healthy; no/low Claims',final_df[i])

    final_df_output = final_df
    final_df_output.drop_duplicates(inplace = True)

    print(final_df_output.head())
    print(final_df_output.groupby('eligible_yr')['health_state'].value_counts())

    print('Output table shape: {}'.format(final_df_output.shape))
#     return final_df_output

  unique_member_id_key    member_id eligible_yr book_of_business_nm  \
0              5626900  20457585004        2018            Americas   
1              5627527  20458010602        2018            Americas   
2              5630202  20457341001        2018            Americas   
3              5635630  20458457301        2018            Americas   
4              5635636  20459090604        2018            Americas   

  health_state_icd10_3digit health_state_icd10_full health_state_ccslv2  \
0              Non-Utilizer            Non-Utilizer        Non-Utilizer   
1              Non-Utilizer            Non-Utilizer        Non-Utilizer   
2              Non-Utilizer            Non-Utilizer        Non-Utilizer   
3              Non-Utilizer            Non-Utilizer        Non-Utilizer   
4              Non-Utilizer            Non-Utilizer        Non-Utilizer   

  health_state_ccslv1         selected_hs_type  health_state  
0        Non-Utilizer  health_state_icd10_full  Non-Utilize